
## Import 

In [5]:
cd ..

/home/alex/Desktop/openclassroom/mentorat/__alexandre_gazagnes_mentorat/DS/P7/for-nora


In [6]:
pwd

'/home/alex/Desktop/openclassroom/mentorat/__alexandre_gazagnes_mentorat/DS/P7/for-nora'

In [ ]:
import os, sys, pickle

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
# from xgboost import XGBClassifier



# import shap

## Load data

In [ ]:
url = "https://gist.githubusercontent.com/AlexandreGazagnes/9018022652ba0933dd39c9df8a600292/raw/0845ef4c2df4806bb05c8c7423dc75d93e37400f/titanic_train_raw_csv"

In [ ]:
df = pd.read_csv (url)

In [ ]:
df.head()

In [ ]:
df.columns

## Select 

In [ ]:
cols = ["PassengerId", "Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
df = df.loc[:, cols]
df.head()

In [ ]:
"""categorical_features = ["Embarked", "Sex"]
for cat in categorical_features : 
    df[cat] = pd.Categorical(df[cat])

df.head()
"""

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.info()

# Export 

In [ ]:
try : 
    os.mkdir("./data/")
except Exception as e : 
    print(e)

In [ ]:
try : 
    os.mkdir("./data/cleaned/")
except Exception as e : 
    print(e)

In [ ]:
df.to_csv("./data/cleaned/df.csv", index=False)

## Prepare

In [ ]:

















STTTTTTOOOOPPPPP



STTTTTTOOOOPPPPP




STTTTTTOOOOPPPPP


STTTTTTOOOOPPPPP




STTTTTTOOOOPPPPP



In [ ]:
X = df.drop(columns="Survived")
y = df.Survived

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(X, y)

## Model

In [ ]:
"""

categorical_features = ["Embarked", "Sex"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder='passthrough'
)

"""

In [ ]:
imputer = KNNImputer().fit(X_train)

X_train = pd.DataFrame(imputer.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)
X_train.info()

In [ ]:
"""

pipe = Pipeline([

    ("imputer",      KNNImputer() ),
    # ("scaler",       "passthrough"),
    ("estimator",    RandomForestClassifier()),
    ])
"""

In [ ]:
grid = GridSearchCV(RandomForestClassifier(), {}, cv=5, return_train_score=True, n_jobs=-1)

## Train

In [ ]:
grid.fit(X_train, y_train)

## Test

In [ ]:
res = pd.DataFrame(grid.cv_results_)
res

In [ ]:
res.columns

In [ ]:
cols = [i for i in res.columns if ("split" not in i) ]
cols

In [ ]:
res = res.loc[:, cols]
res

In [ ]:
grid.score(X_test, y_test)

In [ ]:
model = grid.best_estimator_
model

## Explain

In [ ]:
estimator = model
feat = pd.Series(estimator.feature_importances_, index=X_test.columns)# .sort_values(ascending=False)
px.bar(feat)

In [ ]:
estimator = model
feat = pd.Series(estimator.feature_importances_, index=X_test.columns).sort_values(ascending=False)
px.bar(feat)

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar",show=True)

In [ ]:
"""
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.savefig('grafic.png')
"""

In [ ]:
"""
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
"""

explainer = shap.TreeExplainer(model, X_train)
shap_values = explainer(X_train)
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
shap.plots.waterfall(shap_values[0])

## Save

In [ ]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(grid)
    f.write(pk)